## `**Carregando os dados**`

In [6]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
from google.colab import files
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Fazer upload do arquivo ARFF
uploaded = files.upload()

# O nome do arquivo é a chave no dicionário uploaded
filename = next(iter(uploaded))

# Carregar o arquivo ARFF
data, meta = arff.loadarff(filename)

# Converter para DataFrame do pandas
df = pd.DataFrame(data)

# Exibir os primeiros registros do DataFrame
print(df.head())

# Verificar se há colunas com valores em bytes e convertê-las para string
for col in df.columns:
    if df[col].dtype == 'object':  # Verificando colunas que contêm bytes
        df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# Exibir os primeiros registros do DataFrame após a conversão
print("DataFrame após decodificação:")
print(df.head())

# Identificar as colunas de atributos (X) e a coluna de classe (y)
X = df.iloc[:, :-1]  # Todas as colunas exceto a última (atributos)
y = df.iloc[:, -1]  # Última coluna (classe)

# Aplicar Label Encoding nas colunas categóricas
label_encoder = LabelEncoder()

# Verificar os tipos de dados para identificar colunas categóricas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Aplicar Label Encoding em cada coluna categórica
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Aplicar Label Encoding na coluna de saída (classe), se necessário
y = label_encoder.fit_transform(y)

# Exibir o DataFrame após a transformação
print("DataFrame após Label Encoding:")
print(X.head())



# Normalizar os dados (StandardScaler padroniza os dados com média 0 e desvio padrão 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)  # Normalizando todos os dados antes da validação cruzada


# Exibir os primeiros registros do DataFrame
print(X.view())


Saving breast-cancer.arff to breast-cancer.arff
        age   menopause tumor-size inv-nodes node-caps deg-malig    breast  \
0  b'40-49'  b'premeno'   b'15-19'    b'0-2'    b'yes'      b'3'  b'right'   
1  b'50-59'     b'ge40'   b'15-19'    b'0-2'     b'no'      b'1'  b'right'   
2  b'50-59'     b'ge40'   b'35-39'    b'0-2'     b'no'      b'2'   b'left'   
3  b'40-49'  b'premeno'   b'35-39'    b'0-2'    b'yes'      b'3'  b'right'   
4  b'40-49'  b'premeno'   b'30-34'    b'3-5'    b'yes'      b'2'   b'left'   

   breast-quad irradiat                    Class  
0   b'left_up'    b'no'     b'recurrence-events'  
1   b'central'    b'no'  b'no-recurrence-events'  
2  b'left_low'    b'no'     b'recurrence-events'  
3  b'left_low'   b'yes'  b'no-recurrence-events'  
4  b'right_up'    b'no'     b'recurrence-events'  
DataFrame após decodificação:
     age menopause tumor-size inv-nodes node-caps deg-malig breast  \
0  40-49   premeno      15-19       0-2       yes         3  right   
1  50-5


## **KNN com holdout sem repetição**

In [12]:
# Dividir os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

# Definir o modelo KNN (com k=5, por exemplo)
knn = KNeighborsClassifier(n_neighbors=1)

# Treinar o modelo
knn.fit(X_train, y_train)

# Fazer previsões
y_pred = knn.predict(X_test)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo KNN: {accuracy * 100:.2f}%")

Acurácia do modelo KNN: 63.79%


## `**KNN com holdout com repetições**

In [8]:
# Definir o modelo KNN (com k=5, por exemplo)
knn = KNeighborsClassifier(n_neighbors=1)

# Inicializar listas para armazenar as acurácias
accuracies = []

# Realizar Holdout com 30 repetições
for _ in range(30):
    # Dividir os dados em treino (80%) e teste (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)  # random_state=None para diferentes divisões

    # Treinar o modelo
    knn.fit(X_train, y_train)

    # Fazer previsões
    y_pred = knn.predict(X_test)

    # Avaliar a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calcular a média e o desvio padrão das acurácias
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

# Exibir os resultados
print(f"Acurácia média do Holdout: {mean_accuracy * 100:.2f}%")
print(f"Desvio padrão da acurácia: {std_accuracy * 100:.2f}%")

Acurácia média do Holdout: 72.70%
Desvio padrão da acurácia: 5.79%


## `**KNN com validação cruzada**

In [9]:
# Definir o modelo KNN (com k=5, por exemplo)
knn = KNeighborsClassifier(n_neighbors=1)

# Realizar validação cruzada com 10 grupos
cv_scores = cross_val_score(knn, X, y, cv=10)

# Exibir a acurácia média e desvio padrão da validação cruzada
print(f"Acurácia média da validação cruzada: {cv_scores.mean() * 100:.2f}%")
print(f"Desvio padrão da acurácia: {cv_scores.std() * 100:.2f}%")

Acurácia média da validação cruzada: 69.20%
Desvio padrão da acurácia: 4.82%
